In [1]:
from functional import seq
from functional.streams import Sequence
import pandas as pd
import numpy as np
import os
from typing import Dict, List, Tuple, Optional
import random
from collections import namedtuple
from random import shuffle
import nltk

In [296]:
ustawy_dir = "../lower_ustawy"
art_keyword = "art"
ustawy_files = seq(os.listdir(ustawy_dir)).map(
    lambda filename: open(ustawy_dir + "/" + filename).read())

ustawy_files.size()

1180

In [297]:
good, bad = ustawy_files.partition(lambda x: art_keyword in x[:2400])

In [298]:
good.size()

1178

In [299]:
bad.size()

2

In [300]:
bad[0][:300].split("\n")

['',
 'ustawa',
 'z dnia 11 października 2013 r ',
 'o wzajemnej pomocy przy dochodzeniu podatków  należności ',
 'celnych i innych należności pieniężnych',
 '',
 '     ',
 'font definitions   ',
 '  font face',
 '\t font family helvetica ',
 '\tpanose 1 2 11 5 4 2 2 2 2 2 4  ',
 ' font face',
 '\t font family courier ',
 '\tpanose 1 2 7 4 9 2 2 5 2 4 4  ',
 '']

In [301]:
bad[1][:300].split("\n")

['', '', '', '', '', 'brak tekstu w postaci elektronicznej ', '']

In [302]:
def is_change(ustawa:str) -> bool:
    return "o zmianie ustawy" in ustawa[:800]

In [303]:
changes, not_changes = good.partition(is_change)

In [304]:
Labeled =  namedtuple("Labeled","text is_change")

In [305]:
def strip_title(text:str) -> str:
    return text.split(art_keyword,maxsplit=1)[1]

In [306]:
labeled_changes = changes.map(lambda txt: Labeled(text = strip_title(txt), is_change = True))

In [307]:
labeled_not_changes = not_changes.map(lambda txt: Labeled(text = strip_title(txt), is_change = False))

In [308]:
data = (labeled_changes + labeled_not_changes).to_list()

In [309]:
shuffle(data)

In [310]:
def split(data):
    data_len = len(data)
    train_size = int(0.6 * data_len)
    validation_size = int(0.2 * data_len)

    train = data[:train_size]
    validation = data[train_size:train_size + validation_size]
    test = data[train_size + validation_size:]
    return train,test,validation
    
# train, test,validation = split(data)
# len(train),len(validation),len(test)

In [311]:
from random import choices # todo:bcm - take without replacements
def full_selector(text):
    return text

def percentage_selector(text):
    lines = text.split("\n")
    take_count = int(0.1* len(lines))
    return "\n".join(choices(lines,k= take_count))

def lines_selector(text):
    lines = text.split("\n")
    return "\n".join(choices(lines,k= 10))
    
def line_selector(text):
    lines = text.split("\n")
    lines_len = len(lines)
    return lines[random.randint(0,lines_len-1)]

In [312]:
Selector = namedtuple("Selector", "name selector")

In [313]:
selectors = seq([
    Selector(name = "full", selector = full_selector),
    Selector(name = "percentage", selector = percentage_selector),
    Selector(name = "lines", selector = lines_selector),
    Selector(name = "line", selector = line_selector),
])

## Learning things

In [314]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support

In [315]:
# TODO - use or loose it 
def grid_search(train_x, train_y, test_x, test_y,  parameters, pipeline):
    grid_search_tune = GridSearchCV(
        pipeline, parameters, cv=2, n_jobs=3, verbose=10)
    grid_search_tune.fit(train_x, train_y)

    print("Best parameters set:")
    print(grid_search_tune.best_estimator_.steps)

    print("Applying best classifier on test data:")
    best_clf = grid_search_tune.best_estimator_
    predictions = best_clf.predict(test_x)

    print(classification_report(test_y, predictions))

In [316]:
sw_file = open("polish.stopwords.txt")
stop_words = set(word[:-1] for word in sw_file.readlines())
sw_file.close()
list(stop_words)[:3]

['był', 'poza', 'jezeli']

In [317]:
# nltk.download('punkt')
tokenized = [Labeled(text = nltk.word_tokenize(x.text), is_change= x.is_change) for x in data]

In [318]:
def map_text(mapper, data):
    return [Labeled(text = mapper(x.text), is_change= x.is_change) for x in data]

In [319]:
# train_x = [x.text for x in train]
# y_train = [x.is_change for x in train]
# test_x = [x.text for x in test]
# y_test = [x.is_change for x in test]

In [320]:
def split2(x,y):
    data_len = len(y)
    train_size = int(0.6 * data_len)
    validation_size = int(0.2 * data_len)

    train = x[:train_size]
    validation = x[train_size:train_size + validation_size]
    test = x[train_size + validation_size:]
    
    y_train = y[:train_size]
    y_validation = y[train_size:train_size + validation_size]
    y_test = y[train_size + validation_size:]
    
    
    return (train,test,validation), (y_train,y_test,y_validation)

In [321]:
xs = [x.text for x in data]
ys = [x.is_change for x in data]

In [322]:
from collections import Counter
word_counter = Counter()

for labeled_doc in tokenized:
    for word in labeled_doc.text:
        word_counter[word] += 1
    

In [323]:
words_for_vocab = int(8* len(word_counter))
words_for_vocab

489032

In [324]:
vocab = [word for word,_ in word_counter.most_common(words_for_vocab)]

In [325]:
idf = TfidfVectorizer(stop_words=stop_words,vocabulary=vocab)
idfed = idf.fit_transform(xs)

In [326]:
(tr_x,test_x,val_x),(tr_y,test_y,val_y)= split2(idfed,ys)

In [327]:
from sklearn.linear_model import LogisticRegression

logistic = LogisticRegression()
logistic.fit(tr_x,tr_y)
logistic.score(test_x,test_y)

/home/jacko/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8143459915611815

In [328]:
svc = SVC(kernel= 'linear')#OneVsRestClassifier(SVC(), n_jobs=1)
svc.fit(tr_x,tr_y)
predicted_y = svc.predict(test_x)

In [393]:
def evaluate(predicted,expected):
    prec,recall,fbeta,support = precision_recall_fscore_support(y_true=expected,y_pred = predicted,average='weighted')
   

    print("F1 score: {}".format(fbeta))
    print("Precission: {}".format(prec))
    print("Recall: {}".format(recall))

In [394]:
evaluate(predicted_y,test_y)

F1 score: 0.8351778373912422
Precission: 0.8356614401805458
Recall: 0.8354430379746836


# Fasttext

In [331]:
import fastText

In [332]:
oneline_data = map_text(lambda text: text.replace("\n", " "),data)
oneline_data[0]

Labeled(text='  1   1  ustawa określa stosunek państwa do kościoła ewangelicko metodystycznego    w rzeczypospolitej polskiej  zwanego dalej  kościołem   oraz jego     sytuację prawną i majątkową   2  w sprawach odnoszących się do kościoła  nie uregulowanych w ustawie      stosuje się powszechnie obowiązujące przepisy prawa   3  wszelkie zmiany niniejszej ustawy wymagają uprzedniej opinii rady kościoła                                    art  2   1  kościół wchodzi w skład konferencji generalnej i centralnej zjednoczonego    kościoła metodystycznego oraz jest członkiem światowej rady metodystycznej   2  kościół może należeć do krajowych i międzynarodowych organizacji o charakterze    ekumenicznym i międzywyznaniowym                                     art  3  kościół rządzi się w swoich sprawach własnym prawem wewnętrznym uchwalanym przez konferencję doroczną  swobodnie wykonuje władzę duchowną i  jurysdykcyjną oraz zarządza swoimi sprawami                                     art  4  ko

In [333]:
def to_fast_text(x: Labeled)-> str :
    label = "1" if x.is_change else "0"
    replaced = x.text.replace("\"","\"\"")
    return "__label__{}, \"{}\"\n".format(label,replaced)

In [334]:
data_len = len(oneline_data)
train_size = int(0.6 * data_len)
validation_size = int(0.2 * data_len)

In [335]:
ol_train = oneline_data[:train_size]
ol_test = oneline_data[train_size:train_size+ validation_size]
ol_val = oneline_data[validation_size+ train_size:]

In [336]:
def ol_to_file(name:str, ol:List[Labeled])-> None:
    file = open("data.bak/{}".format(name),"w")

    for t in ol:
        file.write(to_fast_text(t))
    file.close()

In [337]:
ol_to_file("train.csv",ol_train)
ol_to_file("test.csv",ol_test)
ol_to_file("val.csv",ol_val)

In [388]:
model = fastText.train_supervised("data.bak/train.csv",lr=0.7,epoch = 20)

In [389]:
test_result = model.test("data.bak/test.csv")

In [392]:
# TODO:bcm - compute my own precision
test_result

(235, 0.8297872340425532, 0.8297872340425532)

# Flair

In [400]:
ROOT_URL = 'https://s3.eu-central-1.amazonaws.com/borchmann'
FORWARD_FILE = 'cse/lm-polish-forward-v0.2.pt'
BACKWARD_FILE = 'cse/lm-polish-backward-v0.2.pt'
GLOVE_FILE = 'glove/poleval.txt'

FORWARD_LM = f'{ROOT_URL}/{FORWARD_FILE}'
BACKWARD_LM = f'{ROOT_URL}/{BACKWARD_FILE}'
GLOVE = f'{ROOT_URL}/{GLOVE_FILE}'

print(FORWARD_LM)
print(BACKWARD_LM)
print(GLOVE)

https://s3.eu-central-1.amazonaws.com/borchmann/cse/lm-polish-forward-v0.2.pt
https://s3.eu-central-1.amazonaws.com/borchmann/cse/lm-polish-backward-v0.2.pt
https://s3.eu-central-1.amazonaws.com/borchmann/glove/poleval.txt


In [401]:
# ! wget https://s3.eu-central-1.amazonaws.com/borchmann/cse/lm-polish-forward-v0.2.pt -O data.bak/polish_forward.pt
! wget https://s3.eu-central-1.amazonaws.com/borchmann/cse/lm-polish-backward-v0.2.pt -O data.bak/polish_backward.pt
! wget https://s3.eu-central-1.amazonaws.com/borchmann/glove/poleval.txt -O data.bak/glove.txt


--2019-05-12 21:47:00--  https://s3.eu-central-1.amazonaws.com/borchmann/cse/lm-polish-backward-v0.2.pt
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.74.0
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.74.0|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84244196 (80M) [application/x-www-form-urlencoded]
Saving to: ‘data.bak/polish_backward.pt’

data.bak/polish_bac 100%[===================>]  80.34M  3.76MB/s    in 14s     

2019-05-12 21:47:14 (5.86 MB/s) - ‘data.bak/polish_backward.pt’ saved [84244196/84244196]

--2019-05-12 21:47:14--  https://s3.eu-central-1.amazonaws.com/borchmann/glove/poleval.txt
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.74.48
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.74.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 514977227 (491M) [text/plain]
Saving to:

In [ ]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path

corpus = NLPTaskDataFetcher.load_classification_corpus(
    Path('./data.bak'),
    test_file='test.csv',
    dev_file='val.csv',
    train_file='train.csv'
)

word_embeddings = [
    WordEmbeddings('glove'),
    FlairEmbeddings('news-forward-fast'),
    FlairEmbeddings('news-backward-fast')
]

document_embeddings = DocumentLSTMEmbeddings(
    word_embeddings,
    hidden_size=512,
    reproject_words=True,
    reproject_words_dimension=256
)

classifier = TextClassifier(
    document_embeddings,
    label_dictionary=corpus.make_label_dictionary(),
    multi_label=False
)

trainer = ModelTrainer(classifier, corpus)

trainer.train('./data.', max_epochs=10)